In [6]:
# Import necessary modules

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd
from urllib.request import urlretrieve
import requests
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
from datetime import datetime, timedelta
import json
import sys
import math
from geopy.distance import distance
from matplotlib.colors import to_rgba
from collections import defaultdict
import glob
from skimage import exposure
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import netCDF4

In [7]:
# Establish directory locations

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/sat_plot')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')
floats_dir    = os.path.join(parent_dir, 'Data/Floats')
sentinel_dir  = os.path.join(satellite_dir, 'SENTINEL')
sen3_dirs     = [os.path.join(sentinel_dir, d) for d in os.listdir(sentinel_dir) if '.SEN3' in d]


In [ ]:

cur_sen_dir = sen3_dirs[0]

matching_files = []
print(sentinel_dir)
for filename in os.listdir(cur_sen_dir):
    if filename.endswith('.nc'):
        matching_files.append(filename)
        print(filename)
# List of matching files (assuming they end with '_radiance.nc')
matching_files = [file for file in os.listdir(cur_sen_dir) if file.endswith('_radiance.nc')]
#dset = netCDF4.Dataset(cur_sen_dir + '\\' + matching_files[0])
#dset

# Combine all matching files into a single xarray Dataset
olci_xr_mf = xr.open_mfdataset([f'{cur_sen_dir}/{file}' for file in matching_files], combine='by_coords')

# Print the combined Dataset
print(olci_xr_mf)

olci_geo_coords = xr.open_dataset(f'{cur_sen_dir}/geo_coordinates.nc')
olci_geo_coords

lat = olci_geo_coords.latitude.data
lon = olci_geo_coords.longitude.data

def select_channels_for_rgb(xarray, red_channel, green_channel, blue_channel):
    """ 
    Selects the channels / bands of a multi-dimensional xarray for red, green and blue composites.
    
    Parameters:
        xarray(xarray Dataset): xarray Dataset object that stores the different channels / bands.
        red_channel(str): Name of red channel to be selected
        green_channel(str): Name of green channel to be selected
        blue_channel(str): Name of blue channel to be selected

    Returns:
        Three xarray DataArray objects with selected channels / bands
    """  
    return xarray[red_channel], xarray[green_channel], xarray[blue_channel]

red, green, blue = select_channels_for_rgb(olci_xr_mf, 'Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance')

def normalize(array):
    """ 
    Normalizes a numpy array / xarray DataArray object value to values between 0 and 1.
    
    Parameters:
        xarray(numpy array or xarray DataArray): xarray DataArray or numpy array object.

    Returns:
        Normalized array
    """ 
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

redn   = normalize(red)
greenn = normalize(green)
bluen  = normalize(blue)

rgb = np.dstack((redn, greenn, bluen))
rgb = exposure.adjust_gamma(rgb, gamma=0.25)  # gamma < 1 boosts brightness
fig = plt.figure(figsize=(10,6))
plt.imshow(rgb)
rgb = exposure.equalize_adapthist(rgb)
mesh_rgb = rgb[:, :, :]
colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)
colorTuple = np.insert(colorTuple, 3, 1.0, axis=1)

min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66

def visualize_s3_pcolormesh(color_array, array, latitude, longitude, title):
    """ 
    Visualizes a numpy array (Sentinel-3 data) with matplotlib's 'pcolormesh' function as composite.
    
    Parameters:
        color_array (numpy MaskedArray): any numpy MaskedArray, e.g. loaded with the NetCDF library and the Dataset function
        longitude (numpy Array): array with longitude values
        latitude (numpy Array) : array with latitude values
        title (str): title of the resulting plot
    """
    fig=plt.figure(figsize=(20, 12))

    ax=plt.axes(projection=ccrs.Mercator())
    ax.coastlines()
    ax.set_extent([min_lon, max_lon, min_lat, max_lat]) # Mediterranean

    gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, crs=ccrs.PlateCarree())
    gl.xlabels_top=False
    gl.ylabels_right=False
    gl.xformatter=LONGITUDE_FORMATTER
    gl.yformatter=LATITUDE_FORMATTER
    gl.xlabel_style={'size':14}
    gl.ylabel_style={'size':14}
    
    img1 = plt.pcolormesh(longitude, latitude, array*np.nan, color=colorTuple,
                          clip_on = True,
                          edgecolors=None,
                          zorder=0,
                          transform=ccrs.PlateCarree()
                          )
    ax.set_title(title, fontsize=20, pad=20.0)
    plt.show()

In [ ]:
visualize_s3_pcolormesh(color_array=colorTuple,
                        array=red, 
                        latitude=lat, 
                        longitude=lon, 
                        title='Sentinel-3 OLCI Level-1 True Colour - "16 June 2024"')

In [ ]:
def process_sentinel_directory(cur_sen_dir):
    matching_files = [file for file in os.listdir(cur_sen_dir) if file.endswith('_radiance.nc')]

    # Combine all matching files into a single xarray Dataset
    olci_xr_mf = xr.open_mfdataset([os.path.join(cur_sen_dir, file) for file in matching_files], combine='by_coords')

    # Open geo-coordinates file
    olci_geo_coords = xr.open_dataset(os.path.join(cur_sen_dir, 'geo_coordinates.nc'))
    lat = olci_geo_coords.latitude.data
    lon = olci_geo_coords.longitude.data

    def select_channels_for_rgb(xarray, red_channel, green_channel, blue_channel):
        return xarray[red_channel], xarray[green_channel], xarray[blue_channel]

    red, green, blue = select_channels_for_rgb(olci_xr_mf, 'Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance')

    def normalize(array):
        array_min, array_max = array.min(), array.max()
        return (array - array_min) / (array_max - array_min)

    redn = normalize(red)
    greenn = normalize(green)
    bluen = normalize(blue)

    # Replace NaNs and infinite values with 0
    redn = np.nan_to_num(redn, nan=0.0, posinf=0.0, neginf=0.0)
    greenn = np.nan_to_num(greenn, nan=0.0, posinf=0.0, neginf=0.0)
    bluen = np.nan_to_num(bluen, nan=0.0, posinf=0.0, neginf=0.0)

    rgb = np.dstack((redn, greenn, bluen))
    rgb = exposure.equalize_adapthist(rgb)  # Enhance contrast
    rgb = exposure.adjust_gamma(rgb, gamma=0.25)  # Boost brightness
    mesh_rgb = rgb[:, :, :]
    colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)
    colorTuple = np.insert(colorTuple, 3, 1.0, axis=1)

    min_lon = -35
    max_lon = -5
    min_lat = 55
    max_lat = 66

    def visualize_s3_pcolormesh(color_array, array, latitude, longitude, title):
        fig = plt.figure(figsize=(20, 12))
        ax = plt.axes(projection=ccrs.Mercator())
        ax.coastlines()
        ax.set_extent([min_lon, max_lon, min_lat, max_lat])

        gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, crs=ccrs.PlateCarree())
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'size': 14}
        gl.ylabel_style = {'size': 14}

        plt.pcolormesh(longitude, latitude, array * np.nan, color=colorTuple,
                       clip_on=True, edgecolors=None, zorder=0,
                       transform=ccrs.PlateCarree())
        ax.set_title(title, fontsize=20, pad=20.0)
        plt.show()

    visualize_s3_pcolormesh(colorTuple, redn, lat, lon, f"Sentinel-3 Data from {os.path.basename(cur_sen_dir)}")

# Process all directories
for cur_sen_dir in sen3_dirs:
    process_sentinel_directory(cur_sen_dir)

In [8]:
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

def select_channels_for_rgb(xarray, red_channel, green_channel, blue_channel):
    return xarray[red_channel], xarray[green_channel], xarray[blue_channel]

def visualize_s3_pcolormesh(color_array, array, latitude, longitude, title, save_path):
    fig = plt.figure(figsize=(20, 12))
    ax = plt.axes(projection=ccrs.Mercator())
    ax.coastlines()
    min_lon = -35
    max_lon = -5
    min_lat = 55
    max_lat = 66
    ax.set_extent([min_lon, max_lon, min_lat, max_lat])

    gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, crs=ccrs.PlateCarree())
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}

    plt.pcolormesh(longitude, latitude, array * np.nan, color=color_array,
                   clip_on=True, edgecolors=None, zorder=0,
                   transform=ccrs.PlateCarree())
    ax.set_title(title, fontsize=20, pad=20.0)
    plt.savefig(save_path, format='png')
    plt.close()

# List available directories
print("Available directories:")
for i, dir in enumerate(sen3_dirs):
    print(f"{i}: {dir}")

# Select directories to merge
selected_indices = input("Enter the indices of directories to merge, separated by commas: ")
selected_indices = [int(idx) for idx in selected_indices.split(',')]
selected_dirs    = [sen3_dirs[idx] for idx in selected_indices]

# Combine data from all directories
combined_red = []
combined_green = []
combined_blue = []
combined_lat = []
combined_lon = []

for cur_sen_dir in sen3_dirs:
    matching_files = [file for file in os.listdir(cur_sen_dir) if file.endswith('_radiance.nc')]
    olci_xr_mf = xr.open_mfdataset([os.path.join(cur_sen_dir, file) for file in matching_files], combine='by_coords')
    olci_geo_coords = xr.open_dataset(os.path.join(cur_sen_dir, 'geo_coordinates.nc'))
    lat = olci_geo_coords.latitude.data
    lon = olci_geo_coords.longitude.data

    red, green, blue = select_channels_for_rgb(olci_xr_mf, 'Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance')

    combined_red.append(normalize(red))
    combined_green.append(normalize(green))
    combined_blue.append(normalize(blue))
    combined_lat.append(lat)
    combined_lon.append(lon)

# Concatenate the combined data along the appropriate dimensions
combined_red = np.concatenate(combined_red, axis=0)
combined_green = np.concatenate(combined_green, axis=0)
combined_blue = np.concatenate(combined_blue, axis=0)
combined_lat = np.concatenate(combined_lat, axis=0)
combined_lon = np.concatenate(combined_lon, axis=0)

# Replace NaNs and infinite values with 0
combined_red = np.nan_to_num(combined_red, nan=0.0, posinf=0.0, neginf=0.0)
combined_green = np.nan_to_num(combined_green, nan=0.0, posinf=0.0, neginf=0.0)
combined_blue = np.nan_to_num(combined_blue, nan=0.0, posinf=0.0, neginf=0.0)

rgb = np.dstack((combined_red, combined_green, combined_blue))
rgb = exposure.equalize_adapthist(rgb)  # Enhance contrast
rgb = exposure.adjust_gamma(rgb, gamma=0.5)  # Boost brightness
mesh_rgb = rgb[:, :, :]
colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)
colorTuple = np.insert(colorTuple, 3, 1.0, axis=1)

output_path = os.path.join(output_dir, 'composite_image.png')
visualize_s3_pcolormesh(colorTuple, combined_red, combined_lat, combined_lon, "Composite Sentinel-3 Data", output_path)

print('Composite image created successfully.')

Available directories:
0: c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\SENTINEL\S3A_OL_1_ERR____20240616T112644_20240616T121110_20240616T135146_2666_113_294______MAR_O_NR_002.SEN3
1: c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\SENTINEL\S3A_OL_1_ERR____20240616T130743_20240616T135209_20240616T152804_2666_113_295______MAR_O_NR_002.SEN3
2: c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\SENTINEL\S3A_OL_1_ERR____20240617T110032_20240617T114457_20240617T132442_2665_113_308______MAR_O_NR_002.SEN3
3: c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\SENTINEL\S3B_OL_1_ERR____20240616T104753_20240616T113219_20240616T131225_2666_094_151______MAR_O_NR_002.SEN3
4: c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\SENTINEL\S3B_OL_1_ERR____20240616T122852_20240616T131318_20240616T144953_2666_094_152______MAR_O_NR_002.SEN3


C:\Users\hanshil\AppData\Roaming\Python\Python312\site-packages\cartopy\mpl\geoaxes.py:1768: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  result = super().pcolormesh(*args, **kwargs)
